Hugging Face has: Datasets: SetFit/emotion
GoEmotions (Google's emotion dataset)
Sentiment140 (Twitter sentiment)
IMDB Movie Reviews
Mental health specific datasets on Kaggle



In [ ]:
# Install required packages
!pip install datasets pandas matplotlib seaborn wordcloud nltk

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import nltk
from collections import Counter
import re

print("Setup complete! Ready to explore emotional patterns...")

Setup complete! Ready to explore emotional patterns...


In [ ]:
# Load GoEmotions dataset
from datasets import load_dataset
import pandas as pd

print("Loading GoEmotions dataset...")
dataset = load_dataset("go_emotions")

# Convert to pandas for easier exploration
train_df = pd.DataFrame(dataset['train'])
print(f"Training samples: {len(train_df)}")
print("\nFirst few examples:")
print(train_df.head())

Loading GoEmotions dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Training samples: 43410

First few examples:
                                                text labels       id
0  My favourite food is anything I didn't have to...   [27]  eebbqej
1  Now if he does off himself, everyone will thin...   [27]  ed00q6i
2                     WHY THE FUCK IS BAYLESS ISOING    [2]  eezlygj
3                        To make her feel threatened   [14]  ed7ypvh
4                             Dirty Southern Wankers    [3]  ed0bdzj


## Understand the goemotions dataset
Familiarize with the dataset's structure, the different emotions represented, and how they are labeled.


In [ ]:
# Display information about the DataFrame
print("DataFrame Info:")
train_df.info()

# Display column names
print("\nColumn Names:")
print(train_df.columns)

# Display the number of unique labels
print("\nNumber of Unique Labels:")
unique_labels = set(l for sublist in train_df['labels'] for l in sublist)
print(len(unique_labels))
print(unique_labels)

from collections import Counter

label_counts = Counter(l for sublist in train_df['labels'] for l in sublist)
print(label_counts)

id2label = dataset["train"].features["labels"].feature.int2str
label2id = dataset["train"].features["labels"].feature.str2int

sample = dataset["train"][1]
label_ids = sample["labels"]

unique_label_names = [id2label(l) for l in unique_labels]

print("Number of unique labels:", len(unique_label_names))
print("Unique labels:", unique_label_names)



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43410 entries, 0 to 43409
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    43410 non-null  object
 1   labels  43410 non-null  object
 2   id      43410 non-null  object
dtypes: object(3)
memory usage: 1017.6+ KB

Column Names:
Index(['text', 'labels', 'id'], dtype='object')

Number of Unique Labels:
28
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
Counter({27: 14219, 0: 4130, 4: 2939, 15: 2662, 3: 2470, 1: 2328, 7: 2191, 18: 2086, 10: 2022, 20: 1581, 2: 1567, 17: 1452, 6: 1368, 25: 1326, 9: 1269, 22: 1110, 5: 1087, 26: 1060, 13: 853, 11: 793, 8: 641, 14: 596, 24: 545, 12: 303, 19: 164, 23: 153, 21: 111, 16: 77})
Number of unique labels: 28
Unique labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'dis

## Map emotions to binary sentiment
Create a mapping from the 28 GoEmotions labels to a binary sentiment (positive/negative). Some emotions correspond to positive and negative sentiment.


In [ ]:
sentiment_mapping = {
    0: 'positive',   # admiration
    1: 'positive',   # amusement
    2: 'negative',   # anger
    3: 'negative',   # annoyance
    4: 'positive',   # approval
    5: 'positive',   # caring
    6: 'neutral',    # confusion
    7: 'neutral',    # curiosity
    8: 'positive',   # desire
    9: 'negative',   # disappointment
    10: 'negative',  # disapproval
    11: 'negative',  # disgust
    12: 'neutral',   # neutral
    13: 'positive',  # excitement
    14: 'negative',  # fear
    15: 'positive',  # gratitude
    16: 'negative',  # grief
    17: 'positive',  # joy
    18: 'positive',  # love
    19: 'positive',  # optimism
    20: 'positive',  # pride
    21: 'neutral',   # realization
    22: 'positive',  # relief
    23: 'negative',  # remorse
    24: 'negative',  # sadness
    25: 'neutral',   # surprise
    26: 'negative',  # embarrassment
    27: 'neutral'    # no emotion
}

# Print the mapping
print("GoEmotions Label to Binary Sentiment Mapping:")
for label, sentiment in sentiment_mapping.items():
    print(f"Label {label}: {sentiment}")

GoEmotions Label to Binary Sentiment Mapping:
Label 0: positive
Label 1: positive
Label 2: negative
Label 3: negative
Label 4: positive
Label 5: positive
Label 6: neutral
Label 7: neutral
Label 8: positive
Label 9: negative
Label 10: negative
Label 11: negative
Label 12: neutral
Label 13: positive
Label 14: negative
Label 15: positive
Label 16: negative
Label 17: positive
Label 18: positive
Label 19: positive
Label 20: positive
Label 21: neutral
Label 22: positive
Label 23: negative
Label 24: negative
Label 25: neutral
Label 26: negative
Label 27: neutral


## Prepare the data for training
Prepare the data for training by converting the GoEmotions labels to binary sentiment labels, splitting the data into training, validation, and testing sets, and preprocessing the text data.


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

# Download necessary NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Function to determine sentiment from a list of labels
def get_overall_sentiment(labels, sentiment_mapping):
    sentiments = [sentiment_mapping.get(label, 'neutral') for label in labels]
    if 'positive' in sentiments and 'negative' not in sentiments:
        return 'positive'
    elif 'negative' in sentiments and 'positive' not in sentiments:
        return 'negative'
    elif 'positive' in sentiments and 'negative' in sentiments:
        # Prioritize positive if both are present
        return 'positive'
    elif all(s == 'neutral' for s in sentiments):
        return 'neutral'
    else:
        return 'neutral' # Default case for any other combination or empty list

# Apply the sentiment mapping to create the 'sentiment' column
train_df['sentiment'] = train_df['labels'].apply(lambda x: get_overall_sentiment(x, sentiment_mapping))

# Filter out neutral sentiments
binary_df = train_df[train_df['sentiment'] != 'neutral'].copy()

print(f"Original dataset size: {len(train_df)}")
print(f"Binary dataset size (excluding neutral): {len(binary_df)}")
print("\nSentiment distribution in binary dataset:")
print(binary_df['sentiment'].value_counts())

# Split the binary dataset into training, validation, and testing sets (80/10/10)
train_set, temp_set = train_test_split(binary_df, test_size=0.2, random_state=42, stratify=binary_df['sentiment'])
val_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=42, stratify=temp_set['sentiment'])

print(f"\nTraining set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Testing set size: {len(test_set)}")


# Define text preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Tokenize
    tokens = text.split()
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the text column
train_set['processed_text'] = train_set['text'].apply(preprocess_text)
val_set['processed_text'] = val_set['text'].apply(preprocess_text)
test_set['processed_text'] = test_set['text'].apply(preprocess_text)

print("\nFirst few examples of preprocessed text in training set:")
display(train_set[['text', 'processed_text']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Original dataset size: 43410
Binary dataset size (excluding neutral): 26789

Sentiment distribution in binary dataset:
sentiment
positive    18234
negative     8555
Name: count, dtype: int64

Training set size: 21431
Validation set size: 2679
Testing set size: 2679

First few examples of preprocessed text in training set:


,text,processed_text
6178,This is my bestie's favorite song.,besties favorite song
34814,"Thanks! Been clipping her for awhile now, I th...",thanks clipping awhile think im going stick
8811,Is that supposed to be outrageous? Assuming he...,supposed outrageous assuming met requirements ...
34670,"Day 2 Feels good, man!",day 2 feels good man
36705,Maybe its the picture but the fabric looks rea...,maybe picture fabric looks really bad


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Define the class weights
class_weight = {'negative': 1.5656925774400936, 'positive': 0.7345892289130196}

# Define the pipeline
text_classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(class_weight=class_weight))
])

# Train the model
print("Training the model...")
text_classifier.fit(train_set['processed_text'], train_set['sentiment'])
print("Model training complete.")

Training the model...
Model training complete.


## Evaluate the model
Evaluate the trained model's performance on the validation (`val_set`) and test (`test_set`) sets using appropriate metrics.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Evaluate on the validation set
print("Evaluating on the validation set...")
val_predictions = text_classifier.predict(val_set['processed_text'])
print("Validation Set Evaluation:")
print(classification_report(val_set['sentiment'], val_predictions))
print(f"Validation Accuracy: {accuracy_score(val_set['sentiment'], val_predictions):.4f}")

# Evaluate on the test set
print("\nEvaluating on the test set...")
test_predictions = text_classifier.predict(test_set['processed_text'])
print("Test Set Evaluation:")
print(classification_report(test_set['sentiment'], test_predictions))
print(f"Test Accuracy: {accuracy_score(test_set['sentiment'], test_predictions):.4f}")

Evaluating on the validation set...
Validation Set Evaluation:
              precision    recall  f1-score   support

    negative       0.66      0.84      0.74       856
    positive       0.91      0.80      0.85      1823

    accuracy                           0.81      2679
   macro avg       0.79      0.82      0.79      2679
weighted avg       0.83      0.81      0.81      2679

Validation Accuracy: 0.8089

Evaluating on the test set...
Test Set Evaluation:
              precision    recall  f1-score   support

    negative       0.66      0.84      0.74       855
    positive       0.91      0.80      0.85      1824

    accuracy                           0.81      2679
   macro avg       0.79      0.82      0.80      2679
weighted avg       0.83      0.81      0.82      2679

Test Accuracy: 0.8111


The logistic regression model trained with TF-IDF features and class weights achieved an accuracy of approximately 81% on both the validation and test sets. The precision, recall, and F1-score for both the negative and positive classes shows a reasonable performance for this binary sentiment classification task.

## Build a prediction api
Create a web service (using Flask) that loads the saved model and exposes an endpoint that accepts text as input and returns the predicted sentiment.


In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd # Import pandas

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Define preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Handle both string and pandas Series input
    if isinstance(text, str):
        # Process single string
        text = re.sub(r'[^\w\s]', '', text.lower())
        tokens = text.split()
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    elif isinstance(text, pd.Series):
        # Process pandas Series (as originally intended)
        return text.apply(lambda x: ' '.join([
            word for word in re.sub(r'[^\w\s]', '', x.lower()).split()
            if word not in stop_words
        ]))
    else:
        raise TypeError("Input must be a string or a pandas Series")


# Wrap it in a FunctionTransformer
preprocessor = FunctionTransformer(preprocess_text, validate=False)

# Define the class weights (copied from the previous cell)
class_weight = {'negative': 1.5656925774400936, 'positive': 0.7345892289130196}

# Define pipeline
text_classifier = Pipeline([
    ('preprocessor', preprocessor), # Include the preprocessor in the pipeline
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(class_weight=class_weight))
])

# Fit on training data (preprocess text before passing to the pipeline)
text_classifier.fit(train_set['text'], train_set['sentiment'])

# Save pipeline
import joblib
joblib.dump(text_classifier, "sentiment_model.joblib")
print("Model with preprocessing included successfully exported to sentiment_model.joblib")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model with preprocessing included successfully exported to sentiment_model.joblib
